In [4]:
import pandas as pd
import numpy as np
import re

## accessの前処理改善

In [5]:
train = pd.read_csv('../processed_data/train_v7.csv')
test = pd.read_csv('../processed_data/test_v7.csv')
access = pd.concat([train['access'],test['access']]).reset_index(drop=True)

- \t\tは必ずあるのでそこで経路ごとに区切る
- 各経路で'分'の個数をカウントし場合分け
- 欠損はなく、通常１つか２つ
- hun==0の場合は無視できる

In [6]:
def processing_walk_time(access):
    times = []
    min_time = []
    ave_time = []

    for a in access:
        ways = a.split('\t\t')
        time = []
        for way in ways:
            hun = way.count('分')
            if hun == 0:
                continue

            elif hun == 1:
                hoge = 1
                t = re.search(r'\d+分', way).group()[:-1]
                time.append(int(t))

            elif hun == 2:
                if 'バス' not in way:
                    # 9行のみ
#                     print(way)
                    tmp = re.findall(r'徒歩\d+分', way)
                    l = []
                    for s in tmp:
                        t = int(s[2:-1])
                        l.append(int(t))
                    time.append(min(l))

                else:
                    try:
                        b_t = re.search(r'バス\(\d+分\)', way).group()
                    except:
                        b_t = re.search(r'バス\d+分', way).group()
                    b_t = int(re.search(r'\d+', b_t).group())
                    w_t = re.search(r'徒歩\d+分', way).group()
                    w_t = int(re.search(r'\d+', w_t).group())
                    time.append(b_t*5+w_t)

            elif hun == 3:
                # 3行のみ
                if 'バス' in way:
                    t = re.search(r'下車徒歩\d+分', way).group()
                    t = re.search(r'\d+', t).group()
                    time.append(int(t))
                else:
                    tmp = re.findall(r'徒歩\d+分', way)
                    tmp = [int(re.search(r'\d+', t).group()) for t in tmp]
                    time += [int(t) for t in tmp]

            
        times.append(time)
        min_time.append(min(time))
        ave_time.append(np.array(time).mean())

    return times,min_time,ave_time

In [7]:
w_t, min_time , ave_time = processing_walk_time(train['access'])

In [8]:
ave_time

[8.333333333333334,
 11.333333333333334,
 10.0,
 8.666666666666666,
 9.666666666666666,
 7.333333333333333,
 48.333333333333336,
 14.333333333333334,
 8.333333333333334,
 6.333333333333333,
 15.333333333333334,
 11.0,
 13.333333333333334,
 6.0,
 10.666666666666666,
 14.0,
 16.333333333333332,
 19.0,
 6.333333333333333,
 7.666666666666667,
 18.666666666666668,
 6.666666666666667,
 13.0,
 14.0,
 12.333333333333334,
 12.666666666666666,
 7.333333333333333,
 16.333333333333332,
 19.0,
 10.0,
 9.333333333333334,
 14.5,
 17.0,
 21.0,
 7.0,
 18.333333333333332,
 18.0,
 10.0,
 5.0,
 22.666666666666668,
 11.333333333333334,
 10.333333333333334,
 13.333333333333334,
 13.333333333333334,
 19.0,
 13.333333333333334,
 9.666666666666666,
 8.333333333333334,
 6.0,
 11.0,
 19.0,
 8.666666666666666,
 14.0,
 13.666666666666666,
 12.333333333333334,
 11.0,
 4.0,
 5.0,
 12.0,
 11.666666666666666,
 22.666666666666668,
 10.0,
 6.0,
 13.666666666666666,
 10.666666666666666,
 11.666666666666666,
 7.3333333333

In [9]:
for age in train['age']:
    if '年' not in age and '新築' not in age:
        print(age)
        

### 1部屋あたりの面積

In [40]:
train = pd.read_csv('../processed_data/train_v9.csv')
test = pd.read_csv('../processed_data/test_v9.csv')

In [41]:
train.loc[:,['room_num','L','D','K','R','S']]

,room_num,L,D,K,R,S
0,1,0,0,1,0,0
1,1,0,0,0,1,0
2,1,0,0,1,0,0
3,2,1,1,1,0,0
4,2,0,1,1,0,0
...,...,...,...,...,...,...
31462,2,0,1,1,0,0
31463,1,0,0,0,1,0
31464,3,0,1,1,0,0
31465,1,0,1,1,0,0


In [42]:
def are_par_room(row):
    return row['area_num']/row['room_num']

In [43]:
apr = train.apply(are_par_room,axis=1)
apr

0        20.01
1        16.50
2        22.05
3        30.24
4        19.83
         ...  
31462    18.95
31463    12.04
31464    20.00
31465    29.59
31466    25.00
Length: 31467, dtype: float64

In [44]:
train.loc[apr[apr<6].index,['y','access','area_num','room_num']]

,y,access,area_num,room_num
87,48000,半蔵門線\t押上駅\t徒歩6分\t\t都営浅草線\t押上駅\t徒歩6分\t\t京成押上線\t...,4.86,1
2462,45760,日比谷線\t茅場町駅\t徒歩1分\t\t銀座線\t日本橋(東京都)駅\t徒歩5分\t\t半蔵...,5.82,1
7491,184000,西武池袋線\t東長崎駅\t徒歩1分\t\t都営大江戸線\t落合南長崎駅\t徒歩9分,5.83,2
8919,28000,都営三田線\t春日(東京都)駅\t徒歩7分\t\t南北線\t東大前駅\t徒歩8分\t\t丸ノ...,5.20,1
31224,47000,千代田線\t北綾瀬駅\t徒歩9分\t\tつくばエクスプレス\t青井駅\t徒歩16分,4.95,1


In [45]:
apr = test.apply(are_par_room,axis=1)
test.loc[apr[apr<6].index,['access','area_num','room_num']]

,access,area_num,room_num
1698,日比谷線\t茅場町駅\t徒歩1分\t\t銀座線\t日本橋(東京都)駅\t徒歩5分\t\t半蔵...,5.82,1
8483,京王線\t桜上水駅\t徒歩5分\t\t京王線\t上北沢駅\t徒歩10分\t\t東急世田谷線\...,16.00,11
10410,丸ノ内線(池袋－荻窪)\t南阿佐ケ谷駅\t徒歩5分\t\t中央線（快速）\t阿佐ケ谷駅\t徒...,1.00,1
30466,都営三田線\t春日(東京都)駅\t徒歩7分\t\t南北線\t東大前駅\t徒歩8分\t\t丸ノ...,5.20,1


In [48]:
def area_par_room(df):
    def calc(row):
        return row['area_num']/row['room_num']
    
    res = df.apply(calc,axis=1)
    
    return pd.Series(res)

In [52]:
train['area_par_room']=area_par_room(train)

In [57]:
train[train['area_par_room']<8].loc[:,['y','area_num','layout','room_num','area_par_room']]

,y,area_num,layout,room_num,area_par_room
87,48000,4.86,1R,1,4.860
492,40000,7.08,1R,1,7.080
1255,46000,7.70,1R,1,7.700
1636,37000,7.00,1R,1,7.000
2208,40000,7.94,1R,1,7.940
2462,45760,5.82,1R,1,5.820
2599,48500,7.80,1R,1,7.800
7110,49000,7.25,1R,1,7.250
7491,184000,5.83,2LDK,2,2.915
7674,29000,7.01,1R,1,7.010


In [58]:
test['area_par_room'] = area_par_room(test)
test[test['area_par_room']<8].loc[:,['area_num','layout','room_num','area_par_room']]

,area_num,layout,room_num,area_par_room
1698,5.82,1R,1,5.820000
1919,7.83,1R,1,7.830000
2217,7.30,1R,1,7.300000
2803,7.00,1R,1,7.000000
3711,7.34,1R,1,7.340000
4575,7.88,1R,1,7.880000
5381,7.05,1R,1,7.050000
5999,7.20,1R,1,7.200000
6240,7.20,1R,1,7.200000
6371,7.70,1R,1,7.700000
